In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize
import re

In [31]:
df = pd.read_csv('generations/generation_responses/val_ml_excerpts.csv')
df = df[df.response.notna()]

In [32]:
df

,Title,Prompt,response
0,Isolating Sources of Disentanglement in Variat...,I have the following research paper:\nIsolatin...,"I apologize, but I cannot find any explicit re..."
1,Isolating Sources of Disentanglement in Variat...,I have the following research paper:\nIsolatin...,This paper does not contain any information on...
2,Isolating Sources of Disentanglement in Variat...,I have the following research paper:\nIsolatin...,This paper does not include any excerpts relat...
3,Isolating Sources of Disentanglement in Variat...,I have the following research paper:\nIsolatin...,"The provided paper does not discuss privacy, o..."
4,Isolating Sources of Disentanglement in Variat...,I have the following research paper:\nIsolatin...,"There is no mention of progress, advancement o..."
...,...,...,...
6106,Using Pre-Training Can Improve Model Robustnes...,I have the following research paper:\nUsing Pr...,This research paper does not contain qualitati...
6107,Using Pre-Training Can Improve Model Robustnes...,I have the following research paper:\nUsing Pr...,"- ""Our most striking result is that standard p..."
6108,Using Pre-Training Can Improve Model Robustnes...,I have the following research paper:\nUsing Pr...,The provided research paper does not contain a...
6109,Using Pre-Training Can Improve Model Robustnes...,I have the following research paper:\nUsing Pr...,The provided research paper does not contain a...


In [33]:
def sent_tokens(text):
    sents = []
    for line in text.split('\n'):
        if text != '':
            sents.extend(sent_tokenize(line))
    return sents

In [34]:
df['response_sents'] = df.response.map(sent_tokens)
df['num_response_sents'] = df.response_sents.map(len)

In [36]:
def clean_sents(sent_list):
    return [re.sub(r'^\d+\.', '', s.lstrip('- ').replace('*', '')).strip().strip('"') for s in sent_list]

In [37]:
df['cleaned_response_sents'] = df.response_sents.map(clean_sents)

In [39]:
def filter_sents(row):
    return [s for s in row.cleaned_response_sents if s in row.Prompt]

In [40]:
df['filtered_sents'] = df.apply(filter_sents, axis=1)
df['num_filtered_sents'] = df.filtered_sents.map(len)

In [48]:
(df.filtered_sents.map(len) > 0).mean()

0.08793375394321766

In [49]:
df.response = df.filtered_sents.map(lambda l: ' '.join(l))

In [50]:
df['value'] = df.Prompt.map(lambda p: p.split('\n\n')[-1].split(', which refers to')[0][53:])

In [53]:
(df.num_filtered_sents >= 1)[df.num_response_sents > 1].mean()

0.5102974828375286